<a href="https://colab.research.google.com/github/kunakl07/Backend-of-Active-Learning/blob/pre_spec_al/active_learning/active_learning_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Active Learning Notebook

## The training dataset consists of 1394 samples
## The active learning dataset consists of 176 samples
## The retraining dataset would consist of 1570 samples

## The test dataset consists of 201 samples


## Explanation in brief:

### In this notebook, I have taken the model is loaded which performs the predictions on the test data and the uncertian labels(i.e the calls with probabilities between 0.4 to 0.6) are passed to the user labeling.

### Since the model is confident that it has detected a call when the values of prediction is close to zero and it is confident that it has not predicted a call when the value is closer to 1. Therefore the values between 0.4 to 0.6 are the ones where the model is quite uncertian about and hence these calls are passed to the user for labeling.

### Importing the necessary libraries

In [ ]:
import matplotlib
matplotlib.use("Agg")

from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.applications import ResNet152V2
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from keras.optimizers import Adam
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
from keras import models
from keras import layers
from keras import optimizers
from keras.applications import VGG16
from keras.applications import InceptionResNetV2
import sys, numpy
import os
from keras import optimizers, regularizers
from keras.layers import Dropout, Flatten, Dense, Activation, Input
from keras import callbacks, regularizers
from keras.models import load_model
import matplotlib.pyplot as plt 
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.engine import  Model
from keras.layers import Conv2D, GlobalAveragePooling2D
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from keras import models
from keras import layers
from keras import optimizers
from keras.applications import VGG16
from keras.applications import InceptionResNetV2
import sys, numpy
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, regularizers
from keras.models import Sequential
from keras.models import load_model
import matplotlib.pyplot as plt 
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.engine import  Model
from keras.layers import Conv2D, GlobalAveragePooling2D
from sklearn import metrics
import itertools
import tensorflow as tf
import os

from google.colab import files
import datetime, os
%matplotlib inline

Using TensorFlow backend.


### Load the drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Extract the calls and no calls dataset on which predictions would be performed

In [ ]:
!unzip '/content/drive/My Drive/pcen_wav_train_val.zip'

Archive:  /content/drive/My Drive/pcen_wav_train_val.zip
   creating: content/Round2_OS_07_05/
   creating: content/Round2_OS_07_05/train/
   creating: content/Round2_OS_07_05/train/nocalls/
  inflating: content/Round2_OS_07_05/train/nocalls/neg_calls_pod333219_0000.png  
  inflating: content/Round2_OS_07_05/train/nocalls/neg_calls_pod333201_0000.png  
  inflating: content/Round2_OS_07_05/train/nocalls/neg_calls_pod260_0000.png  
  inflating: content/Round2_OS_07_05/train/nocalls/neg_calls_pod2305_0000.png  
  inflating: content/Round2_OS_07_05/train/nocalls/neg_calls_pod333427_0000.png  
  inflating: content/Round2_OS_07_05/train/nocalls/neg_calls_pod2250_0000.png  
  inflating: content/Round2_OS_07_05/train/nocalls/neg_calls_pod2135_0000.png  
  inflating: content/Round2_OS_07_05/train/nocalls/neg_calls_pod333116_0000.png  
  inflating: content/Round2_OS_07_05/train/nocalls/neg_calls_pod2202_0000.png  
  inflating: content/Round2_OS_07_05/train/nocalls/neg_calls_pod2137_0000.png  
  

In [ ]:
!unzip '/content/drive/My Drive/pcen_and_wavelet_test.zip'

Archive:  /content/drive/My Drive/pcen_and_wavelet_test.zip
   creating: content/Round2_OS_07_05/test/
   creating: content/Round2_OS_07_05/test/nocalls/
  inflating: content/Round2_OS_07_05/test/nocalls/calls_pod3_neg21_0000.png  
  inflating: content/Round2_OS_07_05/test/nocalls/calls_pod3_neg46_0000.png  
  inflating: content/Round2_OS_07_05/test/nocalls/calls_pod3_neg33_0000.png  
  inflating: content/Round2_OS_07_05/test/nocalls/calls_pod3_neg42_0000.png  
  inflating: content/Round2_OS_07_05/test/nocalls/calls_pod3_neg88_0000.png  
  inflating: content/Round2_OS_07_05/test/nocalls/calls_pod3_neg26_0000.png  
  inflating: content/Round2_OS_07_05/test/nocalls/calls_pod3_neg12_0000.png  
  inflating: content/Round2_OS_07_05/test/nocalls/calls_pod3_neg54_0000.png  
  inflating: content/Round2_OS_07_05/test/nocalls/calls_pod3_neg82_0000.png  
  inflating: content/Round2_OS_07_05/test/nocalls/calls_pod3_neg7_0000.png  
  inflating: content/Round2_OS_07_05/test/nocalls/calls_pod3_neg6_0

### Enter the data path as well as check the number of files

In [ ]:
train_data_path = '/content/content/Round2_OS_07_05/train/'
validation_data_path='/content/content/Round2_OS_07_05/validation/'
test_data_path = '/content/content/Round2_OS_07_05/test/'
calls_path = '/content/content/Round2_OS_07_05/train/calls'

#Parametres
img_width, img_height = 607, 617

nb_test_samples = sum(len(files) for _, _, files in os.walk(test_data_path))
print(nb_test_samples)
epochs = 100
batch_size = 41
nb_call_samples = sum(len(files) for _, _, files in os.walk(calls_path))
print(nb_call_samples)


201
750


### Load the pretrained model

In [ ]:
model = keras.models.load_model('/content/srkw_100_epochs.h5')


### A thing to note here is that you cannot interchange the tf.keras.models.load_model and keras.models.load_models as there are some functions that are there is tf.keras which are not there is keras and viceversa

In [ ]:
model=tf.keras.models.load_model("/content/srkw_100_epochs.h5")

### Perform the probability prediction on the test dataset

In [ ]:
predictions_probab=model.predict_proba(test_data_generator)

Instructions for updating:
Please use `model.predict()` instead.


### Here is the array of different probability predictions that I got on test dataset and if you see closely the ones in the beginning are

In [ ]:
predictions_probab

array([[0.4358289 ],
       [0.36560276],
       [0.51527685],
       [0.7382855 ],
       [0.7472944 ],
       [0.34923512],
       [0.7985261 ],
       [0.8632383 ],
       [0.69601977],
       [0.67726785],
       [0.7375703 ],
       [0.8394731 ],
       [0.84086776],
       [0.7547285 ],
       [0.867315  ],
       [0.3580198 ],
       [0.54649895],
       [0.23553401],
       [0.22358227],
       [0.6260143 ],
       [0.6533484 ],
       [0.5398861 ],
       [0.33612612],
       [0.5711193 ],
       [0.61178845],
       [0.7094488 ],
       [0.2404327 ],
       [0.7843885 ],
       [0.2964298 ],
       [0.61158234],
       [0.4760732 ],
       [0.74979275],
       [0.734125  ],
       [0.20324193],
       [0.41668478],
       [0.71628034],
       [0.6225189 ],
       [0.54376686],
       [0.7101628 ],
       [0.3538701 ],
       [0.12372316],
       [0.3582008 ],
       [0.50473845],
       [0.6878395 ],
       [0.00946835],
       [0.77455145],
       [0.7491136 ],
       [0.144

### Converting from 2D array to 1D 

In [ ]:
l=[]
l=predictions_probab[:,0]
l

array([0.4358289 , 0.36560276, 0.51527685, 0.7382855 , 0.7472944 ,
       0.34923512, 0.7985261 , 0.8632383 , 0.69601977, 0.67726785,
       0.7375703 , 0.8394731 , 0.84086776, 0.7547285 , 0.867315  ,
       0.3580198 , 0.54649895, 0.23553401, 0.22358227, 0.6260143 ,
       0.6533484 , 0.5398861 , 0.33612612, 0.5711193 , 0.61178845,
       0.7094488 , 0.2404327 , 0.7843885 , 0.2964298 , 0.61158234,
       0.4760732 , 0.74979275, 0.734125  , 0.20324193, 0.41668478,
       0.71628034, 0.6225189 , 0.54376686, 0.7101628 , 0.3538701 ,
       0.12372316, 0.3582008 , 0.50473845, 0.6878395 , 0.00946835,
       0.77455145, 0.7491136 , 0.14483345, 0.59302616, 0.7134563 ,
       0.2129459 , 0.7057595 , 0.30938187, 0.05683732, 0.0800145 ,
       0.22401665, 0.22589347, 0.47530055, 0.09980775, 0.4822474 ,
       0.04867371, 0.34748006, 0.91596097, 0.5844005 , 0.5222191 ,
       0.05832502, 0.61929965, 0.4314097 , 0.47180414, 0.6259652 ,
       0.5147465 , 0.5765604 , 0.5060545 , 0.7629145 , 0.37197

### In this part of the code we are going to identify all the calls that the model is uncertain about and submit it to the user for labeling

In [ ]:
i=0
j=0
confused_calls=[]
for ele in l:
    i=i+1
    if ele>0.4 and ele<0.6:
        confused_calls.append(i)   
        print(i)
        print(ele)

1
0.4358289
3
0.51527685
17
0.54649895
22
0.5398861
24
0.5711193
31
0.4760732
35
0.41668478
38
0.54376686
43
0.50473845
49
0.59302616
58
0.47530055
60
0.4822474
64
0.5844005
65
0.5222191
68
0.4314097
69
0.47180414
71
0.5147465
72
0.5765604
73
0.5060545
77
0.5048085
83
0.5749009
91
0.5659111
94
0.45087895
96
0.5681172
98
0.5837786
111
0.57984865
129
0.44762504
132
0.56983775
135
0.52394813
140
0.41366437
142
0.5860714
150
0.49387667
163
0.44775093
164
0.51117396
169
0.57134473
171
0.5646103
175
0.56798744


## Now we would place the calls that the model is more confident about in the train folder such that the calls predicted by the model could automatically go to the call folder in the training part and the no call predicted by the model would automatically go in the no call folder in the training part.

### Therefore, despite the call actually being call or no call if the model has predicted it as a call then it would go to the call folder for training and if the model has predicted it as no call it would go to the no call folder of the training part.

### Detecting the ids of the calls detected by the model

In [ ]:
import numpy as np
i=0
j=0
pred_call=[]
for ele in l:
    i=i+1
    if ele<0.4:
        pred_call.append(i)   
        print(i)
        print(ele)

2
0.36560276
6
0.34923512
16
0.3580198
18
0.23553401
19
0.22358227
23
0.33612612
27
0.2404327
29
0.2964298
34
0.20324193
40
0.3538701
41
0.12372316
42
0.3582008
45
0.009468346
48
0.14483345
51
0.2129459
53
0.30938187
54
0.056837324
55
0.080014504
56
0.22401665
57
0.22589347
59
0.09980775
61
0.04867371
62
0.34748006
66
0.058325015
75
0.37197798
76
0.25190613
78
0.34614506
79
0.2998662
80
0.023498865
81
0.010232324
82
0.17793101
84
0.06143012
85
0.25314048
86
0.0500152
87
0.24155876
88
0.007952935
92
0.044644378
93
0.37181607
97
0.3618928
149
0.39366245
155
0.30610242
158
0.3480074
168
0.19179873
172
0.39848164


### Detecting the ids of the no calls predicted by the model.

In [ ]:
import numpy as np
i=0
j=0
pred_call_neg=[]
for ele in l:
    i=i+1
    if ele>=0.6:
        pred_call_neg.append(i)   
        print(i)
        print(ele)

4
0.7382855
5
0.7472944
7
0.7985261
8
0.8632383
9
0.69601977
10
0.67726785
11
0.7375703
12
0.8394731
13
0.84086776
14
0.7547285
15
0.867315
20
0.6260143
21
0.6533484
25
0.61178845
26
0.7094488
28
0.7843885
30
0.61158234
32
0.74979275
33
0.734125
36
0.71628034
37
0.6225189
39
0.7101628
44
0.6878395
46
0.77455145
47
0.7491136
50
0.7134563
52
0.7057595
63
0.91596097
67
0.61929965
70
0.6259652
74
0.7629145
89
0.8700721
90
0.83121264
95
0.689564
99
0.81944424
100
0.7034417
101
0.67553747
102
0.69269186
103
0.7103233
104
0.6627126
105
0.7276344
106
0.7232121
107
0.75357574
108
0.7198148
109
0.69121665
110
0.7165594
112
0.87833697
113
0.6971521
114
0.7303747
115
0.7959303
116
0.7702069
117
0.8115676
118
0.6768238
119
0.79120284
120
0.8486508
121
0.76078355
122
0.81134474
123
0.79469806
124
0.74528575
125
0.7460708
126
0.8878975
127
0.78269476
128
0.68210655
130
0.8157509
131
0.83480716
133
0.74557173
134
0.667082
136
0.85389394
137
0.8469033
138
0.6268876
139
0.87261033
141
0.6281994
143
0.60

### Create a list and append the ids of the calls that are to be labelled by the expert

In [ ]:
al_pos_calls=[]
for ele in confused_calls:
    if(ele<88):
        al_pos_calls.append(ele)
al_pos_calls[:]

[1,
 3,
 17,
 22,
 24,
 31,
 35,
 38,
 43,
 49,
 58,
 60,
 64,
 65,
 68,
 69,
 71,
 72,
 73,
 77,
 83]

In [ ]:
import shutil

### Place the calls and no calls in the respective training folders for retraining again on these newly added data from the Active Learning dataset

In [ ]:
onlyfiles = []
j=0
folder_path = "/content/content/Round2_OS_07_05/validation/calls/"
onlyfiles = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
for id, file in enumerate(onlyfiles):
    for j in al_pos_calls:
        if id == j:
            print(file)
            print(id)
            shutil.move(os.path.join(folder_path, file), "/content/content/Round2_OS_07_05/train/calls")

calls67_0000.png
1
calls_pod3_pos321_0000.png
3
calls_pod3_pos314_0000.png
17
calls_pod3_pos309_0000.png
22
calls35_0000.png
24
calls58_0000.png
31
calls_pod3_pos312_0000.png
35
calls59_0000.png
38
calls_pod3_pos323_0000.png
43
calls64_0000.png
49
calls_pod3_pos322_0000.png
58
calls36_0000.png
60
calls_pod3_pos295_0000.png
64
calls77_0000.png
65
calls63_0000.png
68
calls43_0000.png
69
calls_pod3_pos316_0000.png
71
calls49_0000.png
72
calls45_0000.png
73
calls44_0000.png
77
calls56_0000.png
83


In [ ]:
al_neg_calls=[]
for ele in confused_calls:
    if(ele>88):
        al_neg_calls.append(ele)
al_neg_calls[:]

[91, 94, 96, 98, 111, 129, 132, 135, 140, 142, 150, 163, 164, 169, 171, 175]

In [ ]:
onlyfiles = []
j=0
folder_path = "/content/content/Round2_OS_07_05/validation/nocalls/"
onlyfiles = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
for id, file in enumerate(onlyfiles):
    for j in al_neg_calls:
        if id+88 == j:
            print(file)
            print(id)
            shutil.move(os.path.join(folder_path, file), "/content/content/Round2_OS_07_05/train/nocalls")

neg_calls_pod255_0000.png
3
neg_calls_pod214_0000.png
6
neg_calls_pod225_0000.png
8
neg_calls_pod333366_0000.png
10
neg_calls_pod235_0000.png
23
neg_calls_pod333346_0000.png
41
neg_calls_pod216_0000.png
44
neg_calls_pod250_0000.png
47
neg_calls_pod333370_0000.png
52
neg_calls_pod230_0000.png
54
neg_calls_pod333340_0000.png
62
neg_calls_pod333343_0000.png
75
neg_calls_pod333373_0000.png
76
neg_calls_pod248_0000.png
81
neg_calls_pod241_0000.png
83
neg_calls_pod333375_0000.png
87


In [ ]:
pred_call

[2,
 6,
 16,
 18,
 19,
 23,
 27,
 29,
 34,
 40,
 41,
 42,
 45,
 48,
 51,
 53,
 54,
 55,
 56,
 57,
 59,
 61,
 62,
 66,
 75,
 76,
 78,
 79,
 80,
 81,
 82,
 84,
 85,
 86,
 87,
 88,
 92,
 93,
 97,
 149,
 155,
 158,
 168,
 172]

In [ ]:
onlyfiles = []
j=0
folder_path = "/content/content/Round2_OS_07_05/validation/calls/"
onlyfiles = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
for id, file in enumerate(onlyfiles):
    for j in pred_call:
        if id == j:
            print(file)
            print(id)
            shutil.move(os.path.join(folder_path, file), "/content/content/Round2_OS_07_05/train/calls")

calls_pod3_pos299_0000.png
2
calls38_0000.png
6
calls53_0000.png
16
calls_pod3_pos287_0000.png
18
calls46_0000.png
19
calls_pod3_pos284_0000.png
23
calls54_0000.png
27
calls_pod3_pos301_0000.png
29
calls71_0000.png
34
calls_pod3_pos300_0000.png
40
calls42_0000.png
41
calls_pod3_pos285_0000.png
42
calls_pod3_pos308_0000.png
45
calls_pod3_pos317_0000.png
48
calls_pod3_pos282_0000.png
51
calls_pod3_pos292_0000.png
53
calls48_0000.png
54
calls39_0000.png
55
calls50_0000.png
56
calls_pod3_pos286_0000.png
57
calls72_0000.png
59
calls55_0000.png
61
calls57_0000.png
62
calls_pod3_pos324_0000.png
66


In [ ]:
onlyfiles = []
j=0
folder_path = "/content/content/Round2_OS_07_05/validation/nocalls/"
onlyfiles = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
for id, file in enumerate(onlyfiles):
    for j in pred_call:
        if id+88 == j:
            print(file)
            print(id)
            shutil.move(os.path.join(folder_path, file), "/content/content/Round2_OS_07_05/train/calls")

neg_calls_pod333363_0000.png
0
neg_calls_pod252_0000.png
4
neg_calls_pod333374_0000.png
5
neg_calls_pod243_0000.png
9
neg_calls_pod333371_0000.png
61
neg_calls_pod333351_0000.png
67
neg_calls_pod227_0000.png
70


In [ ]:
nb_test_samples = sum(len(files) for _, _, files in os.walk("/content/content/Round2_OS_07_05/validation/calls"))
print(nb_test_samples)

43


In [ ]:
nb_test_samples = sum(len(files) for _, _, files in os.walk("/content/content/Round2_OS_07_05/validation/nocalls"))
print(nb_test_samples)

65


In [ ]:
onlyfiles = []
j=0
folder_path = "/content/content/Round2_OS_07_05/validation/calls"
onlyfiles = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
for id, file in enumerate(onlyfiles):
    shutil.move(os.path.join(folder_path, file), "/content/content/Round2_OS_07_05/train/nocalls/")

In [ ]:
onlyfiles = []
j=0
folder_path = "/content/content/Round2_OS_07_05/validation/nocalls"
onlyfiles = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
for id, file in enumerate(onlyfiles):
    shutil.move(os.path.join(folder_path, file), "/content/content/Round2_OS_07_05/train/nocalls/")

In [ ]:
nb_test_samples = sum(len(files) for _, _, files in os.walk("/content/content/Round2_OS_07_05/validation/nocalls"))
print(nb_test_samples)

0


In [ ]:
nb_test_samples = sum(len(files) for _, _, files in os.walk("/content/content/Round2_OS_07_05/validation/calls"))
print(nb_test_samples)

0


### Here we can see that the data from the active learning dataset has been moved to the train dataset for training

In [ ]:
nb_train_samples = sum(len(files) for _, _, files in os.walk("/content/content/Round2_OS_07_05/train"))
print(nb_train_samples)

1570


### Once the dataset has been moved we are goind to train the model again this new dataset

In [ ]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

train_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)
train_batchsize = 41
train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(img_width, img_height),
    batch_size=train_batchsize,
    class_mode='binary',
shuffle=True)
val_batchsize =32


logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,

    epochs=150,
    )

model.save('AL_srkw_done_100.h5')

Found 1570 images belonging to 2 classes.
Epoch 1/150
38/38 [==============================] - 17s 445ms/step - loss: 0.6159 - accuracy: 0.7351
Epoch 2/150
38/38 [==============================] - 17s 436ms/step - loss: 0.6070 - accuracy: 0.7456
Epoch 3/150
38/38 [==============================] - 17s 435ms/step - loss: 0.6055 - accuracy: 0.7606
Epoch 4/150
38/38 [==============================] - 17s 447ms/step - loss: 0.6003 - accuracy: 0.7522
Epoch 5/150
38/38 [==============================] - 16s 426ms/step - loss: 0.6001 - accuracy: 0.7553
Epoch 6/150
38/38 [==============================] - 16s 434ms/step - loss: 0.5917 - accuracy: 0.7619
Epoch 7/150
38/38 [==============================] - 16s 433ms/step - loss: 0.5827 - accuracy: 0.7711
Epoch 8/150
38/38 [==============================] - 17s 437ms/step - loss: 0.5688 - accuracy: 0.7850
Epoch 9/150
38/38 [==============================] - 16s 425ms/step - loss: 0.5722 - accuracy: 0.7750
Epoch 10/150
38/38 [====================

In [ ]:
model=keras.models.load_model("/content/srkw_100_epochs.h5")

### Check the accuracy of the new model(after active learning)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1. / 255)
from sklearn.metrics import confusion_matrix
import numpy as np

test_generator = ImageDataGenerator()
test_data_generator = test_datagen.flow_from_directory(
    test_data_path,
     target_size=(img_width, img_height),
    batch_size=41,
    shuffle=False)
test_steps_per_epoch = numpy.math.ceil(test_data_generator.samples / test_data_generator.batch_size)


predictions = model.predict_generator(test_data_generator, steps=test_steps_per_epoch)
predictions[predictions<=0.5] = 0
predictions[predictions>0.5] = 1

predicted_classes = (predictions).astype(np.int)

true_classes = test_data_generator.classes
class_labels = list(test_data_generator.class_indices.keys())   

report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

cm = confusion_matrix(true_classes, predicted_classes)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])

# show the confusion matrix, accuracy, sensitivity, and specificity
print(cm)
print("acc: {:.4f}".format(acc))
print("sensitivity: {:.4f}".format(sensitivity))
print("specificity: {:.4f}".format(specificity))

Found 201 images belonging to 2 classes.
              precision    recall  f1-score   support

       calls       0.89      0.77      0.83       101
     nocalls       0.80      0.90      0.85       100

    accuracy                           0.84       201
   macro avg       0.84      0.84      0.84       201
weighted avg       0.84      0.84      0.84       201

[[78 23]
 [10 90]]
acc: 0.8358
sensitivity: 0.7723
specificity: 0.9000


### Load the old model and compare the accuracy of both the models

In [ ]:
model2=tf.keras.models.load_model("/content/srkw_100_epochs.h5")

In [ ]:
test_datagen = ImageDataGenerator(rescale=1. / 255)
from sklearn.metrics import confusion_matrix
import numpy as np

test_generator = ImageDataGenerator()
test_data_generator = test_datagen.flow_from_directory(
    test_data_path,
     target_size=(img_width, img_height),
    batch_size=41,
    shuffle=False)
test_steps_per_epoch = numpy.math.ceil(test_data_generator.samples / test_data_generator.batch_size)


predictions = model2.predict_generator(test_data_generator, steps=test_steps_per_epoch)
predictions[predictions<=0.5] = 0
predictions[predictions>0.5] = 1

predicted_classes = (predictions).astype(np.int)

true_classes = test_data_generator.classes
class_labels = list(test_data_generator.class_indices.keys())   

report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

cm = confusion_matrix(true_classes, predicted_classes)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])

# show the confusion matrix, accuracy, sensitivity, and specificity
print(cm)
print("acc: {:.4f}".format(acc))
print("sensitivity: {:.4f}".format(sensitivity))
print("specificity: {:.4f}".format(specificity))

Found 201 images belonging to 2 classes.
Instructions for updating:
Please use Model.predict, which supports generators.
              precision    recall  f1-score   support

       calls       0.85      0.79      0.82       101
     nocalls       0.80      0.86      0.83       100

    accuracy                           0.83       201
   macro avg       0.83      0.83      0.83       201
weighted avg       0.83      0.83      0.83       201

[[80 21]
 [14 86]]
acc: 0.8259
sensitivity: 0.7921
specificity: 0.8600


In [ ]:
test_data_path="/content/content/Round2_OS_07_05/validation/"

In [ ]:
test_datagen = ImageDataGenerator(rescale=1. / 255)
from sklearn.metrics import confusion_matrix
import numpy as np

test_generator = ImageDataGenerator()
test_data_generator = test_datagen.flow_from_directory(
    test_data_path,
     target_size=(img_width, img_height),
    batch_size=41,
    shuffle=False)
test_steps_per_epoch = numpy.math.ceil(test_data_generator.samples / test_data_generator.batch_size)


predictions = model.predict_generator(test_data_generator, steps=test_steps_per_epoch)
predictions[predictions<=0.5] = 0
predictions[predictions>0.5] = 1

predicted_classes = (predictions).astype(np.int)

true_classes = test_data_generator.classes
class_labels = list(test_data_generator.class_indices.keys())   

report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

cm = confusion_matrix(true_classes, predicted_classes)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])

# show the confusion matrix, accuracy, sensitivity, and specificity
print(cm)
print("acc: {:.4f}".format(acc))
print("sensitivity: {:.4f}".format(sensitivity))
print("specificity: {:.4f}".format(specificity))

Found 176 images belonging to 2 classes.
              precision    recall  f1-score   support

       calls       0.87      0.51      0.64        88
     nocalls       0.65      0.92      0.76        88

    accuracy                           0.72       176
   macro avg       0.76      0.72      0.70       176
weighted avg       0.76      0.72      0.70       176

[[45 43]
 [ 7 81]]
acc: 0.7159
sensitivity: 0.5114
specificity: 0.9205
